In [0]:
#importing important libraries
from pyspark.sql.functions import *

In [0]:
bronze_base_path = "abfss://bronze@lakegenproject2701.dfs.core.windows.net/"
silver_base_path = "abfss://silver@lakegenproject2701.dfs.core.windows.net/"

In [0]:
spark.conf.set(
    "fs.azure.account.key.lakegenproject2701.dfs.core.windows.net",
    "nIzWcPnR1L9tMSj5rW4q4ZGRScBGhZiuZahxB1sTzaeZoBUqNFtf2AUFqFHLgK51PHLhw6s2gNTA+AStj4DGXQ==")

In [0]:
sales_bronze = spark.read.format('parquet').option('header','True').option('inferSchema','true').load(f"{bronze_base_path}sales_data_bronze.parquet")
product_bronze = spark.read.format('parquet').option('header', 'True').option('inferSchema', 'true').load(f"{bronze_base_path}product_data_bronze.parquet")
customer_bronze = spark.read.format('parquet').option('header', 'True').option('inferSchema', 'true').load(f"{bronze_base_path}customer_data_bronze.parquet")
store_bronze = spark.read.format('parquet').option('header', 'True').option('inferSchema', 'true').load(f"{bronze_base_path}store_data_bronze.parquet")


In [0]:
# Casting columns to appropriate types in sales
sales_bronze.printSchema()
sales_bronze=sales_bronze.withColumn("sales_id", col("sales_id").cast("integer"))\
    .withColumn("product_id", col("product_id").cast("integer"))\
        .withColumn("customer_id", col("customer_id").cast("integer"))\
            .withColumn("store_id", col("store_id").cast("integer"))\
                .withColumn('price',col('price').cast('float'))

root
 |-- sales_id: double (nullable = true)
 |-- product_id: double (nullable = true)
 |-- customer_id: double (nullable = true)
 |-- store_id: double (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- sale_date: date (nullable = true)
 |-- ingestion_date: timestamp (nullable = true)
 |-- source_file: string (nullable = true)



In [0]:
sales_bronze.printSchema()
product_bronze.printSchema()
product_bronze=product_bronze.withColumn('product_id',col('product_id').cast('integer'))\
    .withColumn('product_name',col('product_name').cast('string'))\
        .withColumn('category',col('category').cast('string'))\
            .withColumn('price',col('price').cast('float'))

root
 |-- sales_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- store_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- price: float (nullable = true)
 |-- sale_date: date (nullable = true)
 |-- ingestion_date: timestamp (nullable = true)
 |-- source_file: string (nullable = true)

root
 |-- product_id: double (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: double (nullable = true)
 |-- ingestion_date: timestamp (nullable = true)
 |-- source_file: string (nullable = true)



In [0]:
product_bronze.printSchema()
customer_bronze.printSchema()
customer_bronze=customer_bronze.withColumn('customer_id',col('customer_id').cast('int'))\
            .withColumn('phone',col('phone').cast('long'))


root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: float (nullable = true)
 |-- ingestion_date: timestamp (nullable = true)
 |-- source_file: string (nullable = true)

root
 |-- customer_id: double (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone: long (nullable = true)
 |-- ingestion_date: timestamp (nullable = true)
 |-- source_file: string (nullable = true)



In [0]:
customer_bronze.printSchema()
store_bronze.printSchema()
store_bronze=store_bronze.withColumn('store_id',col('store_id').cast('integer'))
store_bronze.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone: long (nullable = true)
 |-- ingestion_date: timestamp (nullable = true)
 |-- source_file: string (nullable = true)

root
 |-- store_id: double (nullable = true)
 |-- store_name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- ingestion_date: timestamp (nullable = true)
 |-- source_file: string (nullable = true)

root
 |-- store_id: integer (nullable = true)
 |-- store_name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- ingestion_date: timestamp (nullable = true)
 |-- source_file: string (nullable = true)



In [0]:
## Silver tables and schema
sales_bronze=sales_bronze.dropna(subset=['sales_id'])
product_bronze=product_bronze.dropna(subset=['product_id'])
customer_bronze=customer_bronze.dropna(subset=['customer_id'])
store_bronze=store_bronze.dropna(subset=['store_id'])

In [0]:
# chnaging column name because having ambiguity
sales_bronze = sales_bronze.withColumnRenamed("quantity", "sales_quantity")
sales_bronze = sales_bronze.withColumnRenamed("price", "sales_price")

In [0]:
# perform the join with qualified columns
sales_silver=sales_bronze.join(customer_bronze,'customer_id','inner')\
    .join(product_bronze,'product_id','inner')\
        .join(store_bronze,'store_id','inner')\
            .filter(col('sales_quantity').isNotNull() & col('sales_price').isNotNull())\
                .withColumn('total_sales_value',col('sales_quantity')*col('sales_price'))\
                    .withColumn('processed_date',current_date())


In [0]:
# Select columns with proper names for the final silver layer dataframe
sales_silver = sales_silver.select(
    col("sales_id"),
    col("product_name"),
    col("category"),
    col("customer_name"),
    col("store_name"),
    col("location"),
    col("sales_quantity").alias("quantity"),
    col("sales_price").alias("price"),
    col("total_sales_value"),
    col("sale_date"),
    col("processed_date")
)
sales_silver.count()


69959

In [0]:
# product_silver
product_silver = sales_silver.groupBy("product_name", "category") \
                            .agg(sum("total_sales_value").alias("total_sales"))
product_silver.display()


product_name,category,total_sales
Product_47,Clothing,179820.05033493042
Product_15,Books,172985.29001808167
Product_214,Books,211784.98052215576
Product_98,Electronics,183132.76099586487
Product_464,Books,164487.58032226562
Product_137,Clothing,193215.7404308319
Product_408,Grocery,174758.33908462524
Product_452,Grocery,192025.02132034302
Product_60,Clothing,169744.84949874878
Product_441,Books,196427.88988780975


In [0]:
# customer_silver
customer_silver = customer_bronze.select(
    "customer_id",
    "customer_name"
)

In [0]:
#customer_type column is not available in the table 
customer_silver.display()


customer_id,customer_name
1,Customer_1
2,Customer_2
3,Customer_3
4,Customer_4
5,Customer_5
7,Customer_7
8,Customer_8
11,Customer_11
12,Customer_12
13,Customer_13


In [0]:
# store_silver
store_silver = store_bronze.select(
    "store_id",
    "store_name",
    "location"
)


In [0]:
store_silver.display()

store_id,store_name,location
1,Store_1,Houston
2,Store_2,Los Angeles
3,Store_3,Los Angeles
4,Store_4,New York
5,Store_5,New York
6,Store_6,Los Angeles
7,Store_7,New York
8,Store_8,New York
9,Store_9,Los Angeles
10,Store_10,Chicago


In [0]:
silver_path = '/path/to/silver/'### Writing in silver file after transformations

# Write Silver layer to Parquet
sales_silver.write.mode("overwrite").parquet(f"{silver_path}sales_silver.parquet")
product_silver.write.mode("overwrite").parquet(f"{silver_path}product_silver.parquet")
customer_silver.write.mode("overwrite").parquet(f"{silver_path}customer_silver.parquet")
store_silver.write.mode("overwrite").parquet(f"{silver_path}store_silver.parquet")


In [0]:
%pip install pymysql

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
jdbc_url = "jdbc:mysql://projectserver2701.mysql.database.azure.com:3306/silver_db"
connection_properties = {
    "user": "aaishamodak",
    "password": "Horcruxes77!",
    "driver": "com.mysql.cj.jdbc.Driver"
}

In [0]:
# Write Silver layer to MySQL
#sales_silver.write.jdbc(url=jdbc_url, table="sales_silver", mode="overwrite", properties=connection_properties)
product_silver.write.jdbc(url=jdbc_url, table="product_silver", mode="overwrite", properties=connection_properties)
customer_silver.write.jdbc(url=jdbc_url, table="customer_silver", mode="overwrite", properties=connection_properties)
store_silver.write.jdbc(url=jdbc_url, table="store_silver", mode="overwrite", properties=connection_properties)

print("Silver layer processing completed successfully.")


Silver layer processing completed successfully.


In [0]:
sales_silver.repartition(7) \
    .write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://projectserver2701.mysql.database.azure.com:3306/silver_db") \
    .option("dbtable", "sales_silver") \
    .option("user", "aaishamodak") \
    .option("password", "Horcruxes77!") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("batchsize", 10000) \
    .mode("append") \
    .save()